In [1]:
import os,sys,glob
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn import preprocessing
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

Using TensorFlow backend.


In [2]:
model = sent2vec.Sent2vecModel()
model.load_model('model/o2o_o2m_Sent2Vec_lower_woParam_0616_768.bin') # sent2vec model
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"\=\_]+\ *" #等號、底線被保留，注意要跟先前一致
# all_df = pd.read_csv('./data/tree-rep-profiles-partial/process2family_df.csv')
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
api_li = [x.lower() for x in api_li] #lowrer case?
len(api_li)

26

In [3]:
encode_dict = {}
for i,v in enumerate(api_li):
    encode_dict[v] = i+1
pickle.dump(file=open('data/tree-rep-profiles_o2o/encode_dict.pkl','wb'),obj=encode_dict)
encode_dict
# encode_dict = pickle.load(open('data/tree-rep-profiles-partial/encode_dict.pkl','rb'))
# encode_dict

{'loadlibrary': 1,
 'createprocess': 2,
 'openprocess': 3,
 'exitprocess': 4,
 'terminateprocess': 5,
 'winexec': 6,
 'createremotethread': 7,
 'createthread': 8,
 'copyfile': 9,
 'createfile': 10,
 'deletefile': 11,
 'regsetvalue': 12,
 'regcreatekey': 13,
 'regdeletekey': 14,
 'regdeletevalue': 15,
 'regqueryvalue': 16,
 'regenumvalue': 17,
 'winhttpconnect': 18,
 'winhttpopen': 19,
 'winhttpopenrequest': 20,
 'winhttpreaddata': 21,
 'winhttpsendrequest': 22,
 'internetopen': 23,
 'internetconnect': 24,
 'httpsendrequest': 25,
 'geturlcacheentryinfo': 26}

## Build Data

In [4]:
test_dir = './data/tree-rep-profiles_o2o/TEST/'

In [5]:
def Sent2vec_emb(root_dir,dev=False,max_length=226,model=model,dim=768,encode_dict=encode_dict,dil=dil,test_dir=test_dir,train_only=False):
    # root_dir = './data/tree-rep-profiles_one2many/'
#     samples_pid_aliases = []
#     processed_pid = []
#     rasmma_dir = next(os.walk(root_dir))[1]
#     samples_pid_valid = valid_pids_df.index.tolist()
#     samples_pid_test = test_pids_df.index.tolist()
    # max_length = 347
    # dim = 768

    dev_pids_path = [] #儲存profile順序?
    
    sent_pad = [0]*dim
    # encode_dict = encode_dict_lower
    all_profiles = []
    all_api_name = []
    all_fam_ans = []
    all_byterep=[]
    hl_rep = []
    sen2vec_length_normalize = []
    sen2vec_dim_normalize = []
    loner = False
    if root_dir.split('/')[-2] in 'EXP': #partial match EXP dir
        loner = True
#     for rasmma in rasmma_dir:
    fam_dir = next(os.walk(root_dir))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir +fam))[1]
        for tree in tree_dir:
            if train_only:
                dest_dir = test_dir + fam + '/' + tree #r檢查此dir是否存在於TEST當中
                if not os.path.exists(dest_dir):
                    continue
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for hl_f in hl_list:
                profile_emb = [] #整個api invocation call
                func_name_emb = [] #僅有api function name
                if not loner:
                    byterep = hl_f.replace('.profile','_byterep.pickle') #拿出hl_f所對應的byterep
                    rep = pickle.load(open(byterep,'rb')) #list type
                    if len(rep) < max_length: #Y1
                        for _ in range(max_length-len(rep)):
                            rep.append(0)
                with open(hl_f,encoding='ISO 8859-1') as f: #X2
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
                lines = re.sub('=\\n','',lines)
                lines = re.sub('y\\n','',lines)
                lines = lines.splitlines()
                if len(lines) > max_length:
                    print("ERR length too long:",hl_f,'=>',len(lines))
                    continue
                dev_pids_path.append(hl_f)
                for line in lines:
                    temp = re.sub(dil," ",line.lower()) # lower? 跟先前一致
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    func_name = temp.split(' ')[0] # X1
                    if func_name not in api_li:
                        print('=ERROR:=',hl_f,'=>',temp)
                    func_name_emb.append(encode_dict[func_name]) # encode証術數數字
                    emb = model.embed_sentence(temp)
                    emb = emb[0]
                    profile_emb.append(emb) #放入embdding，句子=>profile
                    sen2vec_dim_normalize.append(emb) #dim norm用
                sen2vec_length_normalize.append(len(lines))
                if len(lines) < max_length:
                    for _ in range(max_length-len(lines)):
                        profile_emb.append(sent_pad)
                if not loner:
                    hl_rep.append(rep) #Y1
                all_profiles.append(profile_emb) #X2
                all_api_name.append(func_name_emb) #X1
    #             all_byterep.extend(rep_list)
    all_api_name = pad_sequences(all_api_name,maxlen=max_length,padding='post',value=0) #X1

    if not loner:
        return np.array(all_api_name) , np.array(all_profiles) , np.array(hl_rep),np.array(sen2vec_dim_normalize) ,sen2vec_length_normalize, dev_pids_path
    else:
        return np.array(all_api_name) , np.array(all_profiles) ,np.array(sen2vec_dim_normalize) ,sen2vec_length_normalize,dev_pids_path


In [6]:
train_dir = './data/tree-rep-profiles_o2o/normal/'
train_api,train_emb, train_rep, train_normalize , train_len_norm,train_pid_paths =   Sent2vec_emb(train_dir)
print(train_api.shape,train_emb.shape,train_rep.shape)
valid_dir = './data/tree-rep-profiles_o2o/DEV/'
valid_api,valid_emb, valid_rep, valid_normalize , valid_len_norm,valid_pid_paths =   Sent2vec_emb(valid_dir)
print(valid_api.shape,valid_emb.shape,valid_rep.shape)
test_dir = './data/tree-rep-profiles_o2o/TEST/'
test_api,test_emb, test_rep, test_normalize , test_len_norm,test_pid_paths =   Sent2vec_emb(test_dir)
print(test_api.shape,test_emb.shape,test_rep.shape)



  2%|▏         | 1/47 [00:00<00:05,  8.79it/s]

(7215, 226) (7215, 226, 768) (7215, 226)


  2%|▏         | 1/47 [00:00<00:07,  5.87it/s]

(453, 226) (453, 226, 768) (453, 226)


100%|██████████| 47/47 [00:12<00:00,  3.74it/s]


(508, 226) (508, 226, 768) (508, 226)


In [7]:
# 會噴ERR事正常的
small_short_dir = './data/tree-rep-profiles_o2o/small_short/'
small_short_api,small_short_emb, small_short_rep, small_short_normalize , small_short_len_norm,small_short_pid_paths =   Sent2vec_emb(small_short_dir)
print(small_short_api.shape,small_short_emb.shape,small_short_rep.shape)

  0%|          | 0/159 [00:00<?, ?it/s]

ERR length too long: ./data/tree-rep-profiles_o2o/small_short/18.domaiq_0.8/G345/9577445247f0d203e606f22edda7f3de6800649d783b9ad2607bb60c17cf1e08_3248.profile => 376
ERR length too long: ./data/tree-rep-profiles_o2o/small_short/18.domaiq_0.8/G345/9577445247f0d203e606f22edda7f3de6800649d783b9ad2607bb60c17cf1e08_3240.profile => 376
ERR length too long: ./data/tree-rep-profiles_o2o/small_short/18.domaiq_0.8/G329/3920ce44e1ecdb0712266738883f15053f8b46e571b1ca4629c0b1445a5d3a29_3184.profile => 268
ERR length too long: ./data/tree-rep-profiles_o2o/small_short/18.domaiq_0.8/G329/3920ce44e1ecdb0712266738883f15053f8b46e571b1ca4629c0b1445a5d3a29_3224.profile => 268
ERR length too long: ./data/tree-rep-profiles_o2o/small_short/18.domaiq_0.8/G359/d1b6ca346d4ce05bdfcf59bb8acbd6b55ae2267a61ead432e636011dd56e561b_2908.profile => 325
ERR length too long: ./data/tree-rep-profiles_o2o/small_short/18.domaiq_0.8/G359/1c850670c2f3ba12b32b4a896e572fb5e2f202c5b04697bfe053aed4fc9fb268_3304.profile => 272


  1%|▏         | 2/159 [00:00<00:33,  4.68it/s]

ERR length too long: ./data/tree-rep-profiles_o2o/small_short/18.domaiq_0.8/G354/82cd72f730f2b79f37c845654a8c0b5b369dd004af34a7f8e585ca0451be4ee9_3268.profile => 399
ERR length too long: ./data/tree-rep-profiles_o2o/small_short/18.domaiq_0.8/G354/82cd72f730f2b79f37c845654a8c0b5b369dd004af34a7f8e585ca0451be4ee9_3200.profile => 366


  3%|▎         | 4/159 [00:00<00:28,  5.49it/s]

...

 98%|█████████▊| 156/159 [00:38<00:00,  8.06it/s]

ERR length too long: ./data/tree-rep-profiles_o2o/small_short/103.fynloski_0.8/G44/150e9b5d2fc9e45af2cbf3dd0d89d087e7c610f8506ee0499b7daf1b95ed3bf5_3092.profile => 565
ERR length too long: ./data/tree-rep-profiles_o2o/small_short/103.fynloski_0.8/G44/661fd0baa01726208477d594969c273dfe128348f4a37eb403f7c75b2be65812_3300.profile => 544
ERR length too long: ./data/tree-rep-profiles_o2o/small_short/103.fynloski_0.8/G45/090b912680052864ebce185843b1a3a88fbdbc7aa6e002f35b99da3952b08464_3180.profile => 676
ERR length too long: ./data/tree-rep-profiles_o2o/small_short/103.fynloski_0.8/G45/0ab3f6999047bb98f3bc29b8e869f742f3dcf9e7bcbeef261281ead8b65a71e1_3352.profile => 676
ERR length too long: ./data/tree-rep-profiles_o2o/small_short/94.bundlore_0.8/G44/bd266b12436ba846a573fdd3e85dc332a6a210c61c65abda08516c506925b514_3400.profile => 422
ERR length too long: ./data/tree-rep-profiles_o2o/small_short/94.bundlore_0.8/G44/bd266b12436ba846a573fdd3e85dc332a6a210c61c65abda08516c506925b514_3208.profile =

100%|██████████| 159/159 [00:38<00:00,  4.09it/s]

ERR length too long: ./data/tree-rep-profiles_o2o/small_short/121.dialer_0.8/G29/133b0920838fe2b0aaffff6c6bd44d6572fced039a3ff723607f7cd50d4501a8_2852.profile => 1222
ERR length too long: ./data/tree-rep-profiles_o2o/small_short/121.dialer_0.8/G29/107d1cd442e03489aa00997d5fe60434e8929817530fa9927a38cad596da1eae_2836.profile => 1144


(2855, 226) (2855, 226, 768) (2855, 226)


In [8]:
exp_dir = './data/tree-rep-profiles_o2o/EXP/'
exp_api,exp_emb, exp_normalize , exp_len_norm,exp_pid_paths =   Sent2vec_emb(exp_dir)
print(exp_api.shape,exp_emb.shape)

100%|██████████| 45/45 [01:06<00:00,  2.16s/it]


(3893, 226) (3893, 226, 768)


In [9]:
# put small_short into train data?
o2o_api = np.concatenate((train_api,small_short_api),axis=0)
o2o_emb = np.concatenate((train_emb,small_short_emb),axis=0)
o2o_rep = np.concatenate((train_rep,small_short_rep),axis=0)
o2o_normalize = np.concatenate((train_normalize,small_short_normalize),axis=0)
o2o_len_norm = train_len_norm
o2o_len_norm.extend(small_short_len_norm)
o2o_pid_paths = train_pid_paths
o2o_pid_paths.extend(small_short_pid_paths)
assert len(o2o_api) == len(o2o_emb) == len(o2o_rep) == len(o2o_pid_paths) == len(o2o_len_norm)

In [10]:
# phase2 for encoder: train_only=True
train_dir = './data/tree-rep-profiles_o2o/normal/'
train_only_api,train_only_emb, train_only_rep, train_only_normalize , train_only_len_norm,train_only_pid_paths =   Sent2vec_emb(train_dir,train_only=True)
print(train_only_api.shape,train_only_emb.shape,train_only_rep.shape)

100%|██████████| 133/133 [01:39<00:00,  1.33it/s]


(4104, 226) (4104, 226, 768) (4104, 226)


In [14]:
# hl_f = './data/tree-rep-profiles_o2o/EXP/virut/G0/a1be329703e11e79ee94a173700cdece80badbfe5ed04fdfc497b4d141113674_3216.profile'
# with open(hl_f,encoding='ISO 8859-1') as f: #X2
#     lines = f.read()
# lines = re.sub(r'[^\x00-\x7F]+','', lines)
# lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
# lines = re.sub('=\\n','',lines)
# lines = re.sub('y\\n','',lines)
# # lines = lines.splitlines()
# lines

['LoadLibrary#PR@ARB@x-msdownloadransf-er-encodg4b\x05ar@DLL#Ret#N',
 'LoadLibrary#PR@ARB@@DLL#Ret#N',
 'ExitProcess#PR@-2146435052']

## dim normalize

In [11]:
alls = np.concatenate([train_only_normalize,valid_normalize,test_normalize])
mean = np.mean(alls,axis=0)
std = np.std(alls,axis=0)
mean.shape , std.shape

((768,), (768,))

In [12]:
def normalize_dim(emb,length,mean=mean,std=std):
    sent_emb = (emb[:length,:] - mean)/std
    final_emb = np.concatenate([sent_emb, emb[length:,:]],axis=0)
    return final_emb

In [13]:
test_emb_norm = []
valid_emb_norm = []
train_only_emb_norm = [] #pahse2
exp_emb_norm = []
o2o_emb_norm = [] #phase1 for encoder
for emb,length in zip(train_only_emb,train_only_len_norm):
    emb_norm = normalize_dim(emb,length)
    train_only_emb_norm.append(emb_norm)
for emb,length in zip(valid_emb,valid_len_norm):
    emb_norm = normalize_dim(emb,length)
    valid_emb_norm.append(emb_norm)
for emb,length in zip(test_emb,test_len_norm):
    emb_norm = normalize_dim(emb,length)
    test_emb_norm.append(emb_norm)
for emb,length in zip(exp_emb,exp_len_norm):
    emb_norm = normalize_dim(emb,length)
    exp_emb_norm.append(emb_norm)
for emb,length in zip(o2o_emb,o2o_len_norm):
    emb_norm = normalize_dim(emb,length)
    o2o_emb_norm.append(emb_norm)
train_only_emb = np.array(train_only_emb_norm)
valid_emb = np.array(valid_emb_norm)
test_emb = np.array(test_emb_norm)
exp_emb = np.array(exp_emb_norm)
o2o_emb = np.array(o2o_emb_norm)
print(train_only_emb.shape,valid_emb.shape,test_emb.shape,exp_emb.shape,o2o_emb.shape)

(4104, 226, 768) (453, 226, 768) (508, 226, 768) (3893, 226, 768) (10070, 226, 768)


In [14]:
root_dir = './data/tree-rep-profiles_o2o/'
pickle.dump(file=open(root_dir + 'pids_train_only.pkl','wb')
            ,obj=(train_only_api , train_only_emb , train_only_rep),protocol=4)
pickle.dump(file=open(root_dir + 'pids_valid.pkl','wb')
            ,obj=(valid_api,valid_emb,valid_rep),protocol=4)
pickle.dump(file=open(root_dir + 'pids_test.pkl','wb')
            ,obj=(test_api,test_emb,test_rep),protocol=4)
pickle.dump(file=open(root_dir + 'pids_exp.pkl','wb')
            ,obj=(exp_api,exp_emb),protocol=4)
pickle.dump(file=open(root_dir + 'pids_o2o.pkl','wb')
            ,obj=(o2o_api,o2o_emb,o2o_rep),protocol=4)

pickle.dump(file=open(root_dir + 'train_only_pid_paths.pkl','wb'),obj=train_only_pid_paths)
pickle.dump(file=open(root_dir + 'valid_pid_paths.pkl','wb'),obj=valid_pid_paths)
pickle.dump(file=open(root_dir + 'test_pid_paths.pkl','wb'),obj=test_pid_paths)
pickle.dump(file=open(root_dir + 'exp_pid_paths.pkl','wb'),obj=exp_pid_paths)
pickle.dump(file=open(root_dir + 'o2o_pid_paths.pkl','wb'),obj=o2o_pid_paths)